In [2]:
import numpy as np
import pandas as pd
from math import isnan
from datetime import datetime,date
# Saved Tiingo API Token in 'token' variable in apiToken.py file
from apiToken import token

In [3]:
# Getting prices from Tiingo
def getPrice(token,ticker,startDate,endDate):
    return pd.read_csv('https://api.tiingo.com/tiingo/daily/'+ticker+'/prices?startDate='+startDate+'&endDate='+endDate+'&token='+token+'&format=csv')
today = str(date.today())
bndPrice = getPrice(token,'bnd','2019-09-30',today)
bndxPrice = getPrice(token,'bndx','2019-09-30',today)
vtiPrice = getPrice(token,'vti','2019-09-30',today)
vxusPrice = getPrice(token,'vxus','2019-09-30',today)
vtiPrice['returns'] = (vtiPrice.adjClose)/vtiPrice.adjClose.shift(1) - 1
vxusPrice['returns'] = (vxusPrice.adjClose)/vxusPrice.adjClose.shift(1) - 1
bndPrice['returns'] = (bndPrice.adjClose)/bndPrice.adjClose.shift(1) - 1
bndxPrice['returns'] = (bndxPrice.adjClose)/bndxPrice.adjClose.shift(1) - 1
bndPrice = bndPrice.loc[:,['date','returns','adjClose']]
bndxPrice = bndxPrice.loc[:,['date','returns','adjClose']]
vtiPrice = vtiPrice.loc[:,['date','returns','adjClose']]
vxusPrice = vxusPrice.loc[:,['date','returns','adjClose']]

In [4]:
# Reading saved data (Market cap is in billions)
vti = pd.read_csv('Data/vti_MC_data.csv')
vxus = pd.read_csv('Data/vxus_MC_data.csv')
bnd = pd.read_csv('Data/bnd_MC_data.csv')
bndx = pd.read_csv('Data/bndx_MC_data.csv')

In [5]:
df = vtiPrice.rename(columns={"returns":"VTI_Return","adjClose":"VTI_Price"}).merge(vxusPrice.rename(columns={"returns":"VXUS_Return","adjClose":"VXUS_Price"}),on='date',how='outer').merge(bndPrice.rename(columns={"returns":"BND_Return","adjClose":"BND_Price"}),on='date',how='outer').merge(bndxPrice.rename(columns={"returns":"BNDX_Return","adjClose":"BNDX_Price"}),on='date',how='outer').merge(vti.rename(columns={"Market Cap":"VTI_MarketCap"}),on='date',how='left').merge(vxus.rename(columns={"Market Cap":"VXUS_MarketCap"}),on='date',how='left').merge(bnd.rename(columns={"Market Cap":"BND_MarketCap"}),on='date',how='left').merge(bndx.rename(columns={"Market Cap":"BNDX_MarketCap"}),on='date',how='left').sort_values(by='date').reset_index(drop=True)

In [6]:
etfs = ['VTI','VXUS','BND','BNDX']
for etf in etfs:
    list = []
    i = 0
    value = 999
    for mc,ret in zip(df[etf+'_MarketCap'],df[etf+'_Return']):
        if i>0:
            value = (ret+1) * list[i-1]
        list.append(mc) if not isnan(mc) else list.append(value)
        i+=1
    df[etf+'_MarketCap'] = list

In [7]:
df['VTI_weight'] = df['VTI_MarketCap']/(df.VTI_MarketCap+df.VXUS_MarketCap+df.BND_MarketCap+df.BNDX_MarketCap)
df['VXUS_weight'] = df['VXUS_MarketCap']/(df.VTI_MarketCap+df.VXUS_MarketCap+df.BND_MarketCap+df.BNDX_MarketCap)
df['BND_weight'] = df['BND_MarketCap']/(df.VTI_MarketCap+df.VXUS_MarketCap+df.BND_MarketCap+df.BNDX_MarketCap)
df['BNDX_weight'] = df['BNDX_MarketCap']/(df.VTI_MarketCap+df.VXUS_MarketCap+df.BND_MarketCap+df.BNDX_MarketCap)

#Enter starting amount
startingAmount = 10000

portfolioVal = [0 for i in range(len(df))]
portfolioVal[0] = startingAmount
df['Portfolio_Value'] = portfolioVal

In [8]:
i=0
list2 = []
portfolioRet = 1
for vtiR,vxusR,bndR,bndxR,vtiW,vxusW,bndW,bndxW,portfolioValue in zip(df.VTI_Return,df.VXUS_Return,df.BND_Return,df.BNDX_Return,df.VTI_weight.shift(1),df.VXUS_weight.shift(1),df.BND_weight.shift(1),df.BNDX_weight.shift(1),df.Portfolio_Value):
    if i>0:
        portfolioRet = (vtiR*vtiW+vxusR*vxusW+bndR*bndW+bndxR*bndxW)+1
    list2.append(portfolioValue) if portfolioValue != 0 else list2.append(list2[i-1]*portfolioRet)
    i+=1
df['Portfolio_Value'] = list2

In [9]:
df

,date,VTI_Return,VTI_Price,VXUS_Return,VXUS_Price,BND_Return,BND_Price,BNDX_Return,BNDX_Price,VTI_MarketCap,VXUS_MarketCap,BND_MarketCap,BNDX_MarketCap,VTI_weight,VXUS_weight,BND_weight,BNDX_weight,Portfolio_Value
0,2019-09-30,NaN,148.218599,NaN,50.350477,NaN,81.984829,NaN,56.800093,29401.430000,999.000000,22512.530000,19421.010000,0.406468,0.013811,0.311230,0.268491,10000.000000
1,2019-10-01,-0.013046,146.284886,-0.009098,49.892391,0.000967,82.064120,-0.001668,56.705377,29017.848430,989.911150,22534.302867,19388.625144,0.403414,0.013762,0.313278,0.269546,9944.247162
2,2019-10-02,-0.016238,143.909462,-0.015628,49.112670,0.002253,82.249015,0.000000,56.705377,28546.646704,974.440767,22585.073856,19388.625144,0.399283,0.013630,0.315898,0.271189,9883.984865
3,2019-10-03,0.008117,145.077543,0.007740,49.492784,0.003313,82.521491,0.003749,56.917974,28778.353338,981.982578,22659.894262,19461.315968,0.400358,0.013661,0.315239,0.270741,9937.453474
4,2019-10-04,0.013058,146.971992,0.006696,49.824166,0.001651,82.657729,0.000170,56.927638,29154.146450,988.557491,22697.304465,19464.620096,0.403213,0.013672,0.313912,0.269203,9995.943607
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,2020-12-02,0.000846,189.230000,0.000681,58.740000,-0.001703,87.930000,0.000172,58.290000,36989.283980,27711.935425,23758.292241,22234.472490,0.334158,0.250347,0.214630,0.200864,11425.406609
298,2020-12-03,0.002008,189.610000,0.004597,59.010000,0.001933,88.100000,0.002402,58.430000,37063.563576,27839.314086,23804.225480,22287.874894,0.333921,0.250816,0.214462,0.200801,11456.474040
299,2020-12-04,0.010021,191.510000,0.009151,59.550000,-0.002611,87.870000,-0.000685,58.390000,37434.961555,28094.071409,23742.080510,22272.617064,0.335608,0.251866,0.212850,0.199676,11513.114159
300,2020-12-07,-0.001097,191.300000,-0.005038,59.250000,0.001366,87.990000,0.001713,58.490000,37393.912305,27952.539563,23774.503973,22310.761639,0.335577,0.250849,0.213355,0.200219,11501.552603
